In [1]:
import sys
sys.path.append("/Users/david/Documents/code/fhmm/v1")

In [5]:
# test for gmm-ubm on whale calls
from importlib import reload

import src.ads_davidjames9610.ads as myads

from collections import Counter
import numpy as np
import librosa
import matplotlib.pyplot as plt
import os.path

In [12]:
# create ads
# female ads
datasetFolder = r"/Users/david/Documents/data/speech/ivectors/SPEECH DATA/FEMALE/MIC"
female_ads = myads.AudioDatastore()
female_ads.populate(datasetFolder, include_sub_folders=True, label_source=True)
Counter(female_ads.labels).values()
# male ads
datasetFolder = r"/Users/david/Documents/data/speech/ivectors/SPEECH DATA/MALE/MIC"
male_ads = myads.AudioDatastore()
male_ads.populate(datasetFolder, include_sub_folders=True, label_source=True)
Counter(male_ads.labels).values()
# combine ads
ads = myads.AudioDatastore()
ads.set([datasetFolder], files=female_ads.files + male_ads.files, labels=female_ads.labels + male_ads.labels)


In [13]:
# set up vad
import torch
torch.set_num_threads(1)
model, torch_utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True)
(get_speech_timestamps,
 _, read_audio,
 *_) = torch_utils

KeyboardInterrupt: 

In [25]:
import src.misc_davidjames9610.proc_methods as proc_methods
reload(proc_methods)

samples = []
labels = []
sr = 16000
# SAMPLING_RATE = 16000

labels_set = list(set(ads.labels))
num_to_label = {i: labels_set[i] for i in range(len(labels_set))}
label_to_num = {labels_set[i] : i for i in range(len(labels_set))}

for i in range(len(ads.files)):

    if i % 10 == 0:
        print(i, '/', len(ads.files))

    labels.append(ads.labels[i])

    signal = proc_methods.file_to_audio(ads.files[i], sr)

    speech_timestamps = get_speech_timestamps(signal, model, sampling_rate=sr)
    segments = []
    for i in range(len(speech_timestamps)):
        start = speech_timestamps[i]['start']
        end = speech_timestamps[i]['end']
        segments.append(signal[start: end])
    segments_flattened = np.array([item for sublist in segments for item in sublist])
    samples.append(segments_flattened)

    # plt.plot(signal)
    # plt.title('before vad')
    # plt.show()
    # plt.close()

    # plt.plot(segments_flattened)
    # plt.title('after vad')
    # plt.show()
    # plt.close()


0 / 4718
10 / 4718
20 / 4718
30 / 4718
40 / 4718
50 / 4718
60 / 4718
70 / 4718
80 / 4718
90 / 4718
100 / 4718
110 / 4718
120 / 4718
130 / 4718
140 / 4718
150 / 4718
160 / 4718
170 / 4718
180 / 4718
190 / 4718
200 / 4718
210 / 4718
220 / 4718
230 / 4718
240 / 4718
250 / 4718
260 / 4718
270 / 4718
280 / 4718
290 / 4718
300 / 4718
310 / 4718
320 / 4718
330 / 4718
340 / 4718
350 / 4718
360 / 4718
370 / 4718
380 / 4718
390 / 4718
400 / 4718
410 / 4718
420 / 4718
430 / 4718
440 / 4718
450 / 4718
460 / 4718
470 / 4718
480 / 4718
490 / 4718
500 / 4718
510 / 4718
520 / 4718
530 / 4718
540 / 4718
550 / 4718
560 / 4718
570 / 4718
580 / 4718
590 / 4718
600 / 4718
610 / 4718
620 / 4718
630 / 4718
640 / 4718
650 / 4718
660 / 4718
670 / 4718
680 / 4718
690 / 4718
700 / 4718
710 / 4718
720 / 4718
730 / 4718
740 / 4718
750 / 4718
760 / 4718
770 / 4718
780 / 4718
790 / 4718
800 / 4718
810 / 4718
820 / 4718
830 / 4718
840 / 4718
850 / 4718
860 / 4718
870 / 4718
880 / 4718
890 / 4718
900 / 4718
910 / 4718

In [2]:
import pickle
# pickle.dump({
#     'samples': samples,
#     'labels': labels,
#     'labels_set': labels_set,
#     'num_to_label': num_to_label,
#     'label_to_num': label_to_num
# }, open('samples_labels.pickle','wb'))

In [7]:
sls_a = pickle.load(open('samples_labels.pickle','rb'))

sls_b = pickle.load(open('/Users/david/Documents/code/fhmm/v1/src/classifiers_davidjames9610/test_a/samples_labels.pickle','rb'))

In [9]:
new_labels = []
for label in sls_a['labels']:
    new_labels.append(sls_a['label_to_num'][label])

In [10]:
sls_a['labels'] = new_labels

In [11]:
pickle.dump(sls_a, open('samples_labels.pickle','wb'))